In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 


select app_id, est_average_active_users from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,est_average_active_users
    from mu.city_app_m
    where 
        date between '2018-09-01' and '2020-09-05' and est_average_active_users is not null and est_average_active_users <> 0
    group by date
\$proxy\$) tpl (date DATE , count_a BIGINT, sum_a REAL) group by date order by date asc;

EOF

In [0]:

from pyspark.sql import functions as F

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/date=2020-05-30/").filter('metric_name="est_average_active_users" and date between "2018-09-01" and "2020-09-05"').groupBy('date').agg(F.count("metric_name").alias("count"),F.sum("value").alias("sum")).show()

In [0]:


from applications.db_check_v1.common.utils import get_date_list
from pyspark.sql import functions as F

from datetime import datetime

begin_date = datetime(2018, 9, 1)
end_date = datetime(2020, 9, 5)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


date_list = DATE_GRANULARITY_MAPPINGLIST["weekly"]

result_list = []
for date_str in DATE_GRANULARITY_MAPPINGLIST["weekly"]:
    data_agg = spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/date={}/".format(date_str)).filter('metric_name="est_average_active_users" and value is not null and value <> 0').agg(F.count("metric_name").alias("count"),F.sum("value").alias("sum")).collect()
    print "{}|{}|{}".format(date_str,data_agg[0]["count"], data_agg[0]["sum"])
    result_list.append([date_str,data_agg[0]["count"], data_agg[0]["sum"]])
print "**weekly**"*10
print result_list
print "**weekly**"*10



In [0]:

data_agg[0]["count"]

In [0]:
%%sh

aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/

In [0]:


raw_data_path =  "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2020-08-16/"
spark.read.parquet(raw_data_path).createOrReplaceTempView("test")
print(spark.sql("select* from test where app_id='20600013770959' ").show(5))
print(spark.sql("select* from test where app_id='284882215' ").show(5))


In [0]:

%sh

PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;



--select country_code,count(country_code) from usage_basic_kpi_fact_v7 where granularity='daily' and date='2020-08-16' and est_average_active_users <> 0  and est_average_active_users is not null and country_code == 'CA' group by country_code order by count(country_code) desc;
    
--select distinct(app_id) from usage_basic_kpi_fact_v7 where granularity='daily' and date='2020-08-16' and est_average_active_users <> 0  and est_average_active_users is not null and country_code = 'CA' order by app_id desc ;

select app_id, country_code,device_code,est_average_active_users  from usage_basic_kpi_fact_v7 where app_id=20600013770959 and date = '2020-08-16' order by date desc limit 100;

EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
select  app_id,kpi,estimate from plproxy.execute_select(\$proxy\$
    SELECT app_id,kpi,estimate FROM mu.app_daily where date ='2020-08-16' and kpi = 1 and app_id=20600013770959 and estimate is not null and estimate <> 0 \$proxy\$) t (app_id bigint,kpi smallint,  estimate double precision) ;


EOF

In [0]:
%%sh
